## Top 10 arrival airports in the world in 2013 (using the bookings file)

Arrival airport is the column arr_port. It is the IATA code for the airport

To get the total number of passengers for an airport, you can sum the "pax" column, grouping by arr_port.

Note that there is negative pax. That corresponds to cancelations. So to get the total number of passengers that have actually booked, you should sum including the negatives (that will remove the canceled bookings).

Print the top 10 arrival airports in the standard output, including the number of passengers.

Bonus point: Get the name of the city or airport corresponding to that airport (programatically, we suggest to have a look at GeoBases in Github)

Bonus point: Solve this problem using pandas (instead of any other approach)


Suggestion: follow the below plan of action:

* Get familiar with the data
* Select columns of interest
* Decide what to do with NaNs

* Make processing plan
* Develop code that works with a sample

* Adjust the code to work with Big data
* Test big data approach on a sample

* Run program with big data


## 1) Get familiar with data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('ggplot')

file = '../../../../Data/challenge/bookings.csv'


In [3]:
cd /home/dsc/Repositories/master-data-science/practice/JupyterChallenge


/home/dsc/Repositories/master-data-science/practice/JupyterChallenge


In [17]:
!bzcat ./../../../../Data/challenge/bookings.csv.bz2 | head -n 100000 > bookings.sample.csv


bzcat: I/O or other error, bailing out.  Possible reason follows.
bzcat: Broken pipe
	Input file = ./../../../../Data/challenge/bookings.csv.bz2, output file = (stdout)


In [2]:
!ls  -lah bookings.sample.csv

-rw-rw-r-- 1 dsc dsc 41M Jan 26 21:10 bookings.sample.csv


### What if we dont want to read the whole file?

Options:

* prepare the sample

* read_csv with nrows option

In [20]:
sample_df = pd.read_csv(file, sep='^',skip_blank_lines=True,nrows=10000)
sample_df.shape

(10000, 38)

Clean the column names

In [21]:
# le quitamos los espacios.
sample_df.columns = sample_df.columns.str.strip()


## 2) Select the columns of interest 

In [23]:
sample_df['arr_port']
sample_df['off_time'].iloc[0][:4]
sample_cols = sample_df[['arr_port','off_time','pax']]
sample_cols.head()

,arr_port,off_time,pax
0,LHR,2013-03-07 11:33:37,-1
1,CLT,2013-04-12 22:05:40,1
2,CLT,2013-07-15 11:34:51,1
3,SVO,2013-04-25 16:06:31,1
4,SVO,2013-05-16 10:44:50,1


 3) What to do with NaN?



In [24]:
sample_cols.isnull().sum()

arr_port    0
off_time    0
pax         0
dtype: int64

In [ ]:
nulls_per_airport = sample_cols.groupby('arr_port').apply(lambda df : df.isnull.sum())
nulls_per_airport.head()

In the sample everything might be ok, but we should prepare for NaN case

## 4) Make processing plan
1) get only the bookings from 2013

2) group by arr_port, sum

3) sort 

4) get top 10

#### 4.1) Get only the booking from 2013

In [29]:
arrivals_2013 = sample_cols['off_time'].str[:4] == '2013'
sample_cols = sample_cols[arrivals_2013]

#### 4.2) group by arr_port, sum

In [31]:
pax_per_airport = sample_cols.groupby('arr_port')['pax'].sum()

In [ ]:
# 4.3,4)sort,get top10

In [ ]:
pax_per_airport.sort_values(ascending=False).head(10)

## 5) Adjust the code to work with Big data


Hint: check out https://pandas.pydata.org/pandas-docs/stable/io.html#io-chunking

In [8]:
chunks = pd.read_csv(file, sep='^',skip_blank_lines=True,nrows=10000000,chunksize=1000000)

### We have to read the whole file, but with nrows we are reading always the first N rows


In [9]:
partial_results =[]
chunk_num= 0
for chunk in chunks:
    chunk.columns = chunk.columns.str.strip()
    chunk =chunk[['arr_port','off_time', 'pax']]
    chunk = chunk [chunk['off_time'].str[:4] == '2013']
    
    partial_results = chunk.groupby('arr_port')['pax'].sum()
    
    partial_results.append(partial_results)
    print(chunk_num)
    chunk_num +=1
    
partial_results

0
1
2
3
4


/home/dsc/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (7,8,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


5
6
7
8
9


arr_port
AAB            3
AAE           81
AAL           75
AAN            2
AAQ          165
AAR           50
ABA            9
ABB            4
ABE           83
ABI           15
ABJ          250
ABK            0
ABQ          404
ABR            0
ABS           31
ABT           29
ABV          188
ABX           30
ABY            9
ABZ          358
ACA           69
ACC          353
ACE          155
ACH            2
ACI            4
ACK           20
ACT           16
ACV           14
ACY           19
ADA           98
            ... 
ZAZ            5
ZBF           11
ZBL            9
ZCL           15
ZCO           58
ZDH            4
ZFQ            4
ZFV           12
ZHA            0
ZIG            4
ZIH           68
ZLN            0
ZLO           49
ZNE          138
ZNZ           94
ZOS            4
ZQN          136
ZRH         1905
ZSA            1
ZSE            1
ZTH            5
ZUH            4
ZVE            4
ZVJ            2
ZVK            2
ZWE            3
ZWS           57
ZYL  

In [ ]:
s = pd.concat(partial_results)
s.groupby('arr_port').sum().sort_values(ascending=False).head()

#### Now we need to put together the results from all the chunks

Options:

* df.append()

* pd.concat()
    

#### 1) df.append()

#### 2) pd.concat()

## Complete Solution